In [3]:
%run sandbox.ipynb

Loading credentials from token.json


In [5]:
main()

Anki sync successful.
Total cards found in 'Pleco Import': 1659
success
Latest flashcard xml last modified time: 2025-01-02 12:17:32 EST-0500
1648 flashcard entries found| 1 error entries found


In [2]:
import json

with open('flashcard_entries.json', 'r') as file:
    flashcard_entries = json.load(file)

flashcard_entries[0]

{'simplified': '艘',
 'traditional': '艘',
 'pinyin': 'sōu',
 'definition': 'measure word [for boats or ships] 兩艘油船 Liǎng sōu yóuchuán two tankers 一艘飛船 Yī sōu fēichuán a spaceship 四艘巡洋艦 sì sōu xúnyángjiàn four cruisers',
 'dictid': 'PACE',
 'formatted_back': '<div align="left"><p><span style="font-size:32px";>艘</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#E30000;"><span style="font-weight:600;">sōu</span></span></p>\n</div><div align="left"><p><b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">MEASURE WORD</span></span></b><br>\n[for boats or ships]<br>\n</p>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">兩</span><span style="color:#0078C3;"><b>艘</b></span><span style="color:#0078C3;">油船</span><br>\n<span style="font-weight:600;">Liǎng </span><b>sōu</b><span style="font-weight:600;"> yóuchuán</

In [14]:
def fmt_entry(entry):
    traditional = entry.get("traditional", "")
    simplified = entry.get("simplified", "")
    pinyin = entry.get("pinyin", "")
    definition = entry.get("definition", "")
    simplified_hint = f"〔{simplified}〕" if traditional != simplified else ""

    formatted_back = ""
    formatted_back += f'<div align="left"><p><span style="font-size:32px";>{traditional}{simplified_hint}</span><br>\n'

    return formatted_back

In [28]:
def grade_fmt_entry(flashcard_entries, n_error_char_show=10):
    correct_count = 0
    length_diff_count = 0
    for i, entry in enumerate(flashcard_entries):
        expected = entry['formatted_back']
        expected = expected.replace(' ;=""', ";")
        result = fmt_entry(entry)
        
        if expected != result:
            for j in range(min(len(expected), len(result))):
                if expected[j] != result[j]:
                    print(f"Entry {i} differs at character {j}:")
                    print(f"Exp: {repr(expected[j:j+n_error_char_show])}...")
                    print(f"Got: {repr(result[j:j+n_error_char_show])}...")
                    break
                    # return
            else:
                length_diff_count += 1
                continue
        else:
            correct_count += 1
    print(f"Total correct entries: {correct_count}")
    print(f"Total entries differing only in length: {length_diff_count}")

In [29]:
grade_fmt_entry(flashcard_entries, 100)

Entry 376 differs at character 52:
Exp: '〔于〕</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span>'...
Got: '</span><br>\n'...
Entry 752 differs at character 52:
Exp: '</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><sp'...
Got: '〔闲〕</span><br>\n'...
Total correct entries: 0
Total entries differing only in length: 1646


In [27]:
flashcard_entries[376]

{'simplified': '於',
 'traditional': '於',
 'pinyin': 'wū',
 'definition': 'interjection classical ah! oh! \nnoun archaic crow 於鵲 wūquè crow and magpie',
 'dictid': 'PACE',
 'formatted_back': '<div align="left"><p><span style="font-size:32px";>於〔于〕</span><br>\n<span style="color:#B4B4B4;"><b><span style="font-size:0.80em;">PY </span></b></span><span style="color:#02B31C;"><span style="font-weight:600;">yú</span></span></p>\n</div><div align="left"><p><b><span style="font-size:0.80em;"><span style="color:#B4B4B4;">PREPOSITION</span></span></b><br>\n<b>1\t</b>[indicating time or place] in; on; at<br>\n</p>\n<blockquote style="border-left: 2px solid #0078c3; margin-left: 3px; padding-left: 1em; margin-top: 0px; margin-bottom: 0px;"><p><span style="color:#0078C3;">生</span><span style="color:#0078C3;"><b>於</b></span><span style="color:#0078C3;">上海</span><br>\n<span style="font-weight:600;">shēng</span><b>yú</b><span style="font-weight:600;"> Shànghǎi</span><br>\nborn in Shanghai<br>\n</p>\n</